## Master ML Pipeline

In [1]:
# Loading the key packages
import time
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import sklearn
import imblearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import sklearn.linear_model as linear_model
import sklearn.calibration as cal
import xgboost as xgb
import sklearn.utils.class_weight as wt
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn import preprocessing
from pathlib import Path
from sklearn import datasets, metrics, model_selection, svm
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn.model_selection import GridSearchCV
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import LinearSVC
from sklearn.svm import LinearSVR
from sklearn.dummy import DummyClassifier
from sklearn.dummy import DummyRegressor
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold

import skopt
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

import multiprocessing



from tensorflow.keras import models, layers, utils, backend as K, callbacks
import matplotlib.pyplot as plt
import shap
import tensorflow
tensorflow.compat.v1.disable_v2_behavior() 
import random
import os

import warnings
#warnings.filterwarnings('ignore')


from numpy.ma import MaskedArray
import sklearn.utils.fixes

sklearn.utils.fixes.MaskedArray = MaskedArray


2024-10-05 15:51:41.454816: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


In [5]:
print(sklearn.__version__)

1.5.2


In [3]:
pip install scikit-learn==1.1.1 --user

  Using cached scikit-learn-1.1.1.tar.gz (6.8 MB)
  Installing build dependencies ... -^C
anceled
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [12]:
# %% Initializing dataframes to store the results


all_results = []


# hyperparam_results = pd.DataFrame(
#     {
#         "Procedure": [],
#         "Outcome": [],
#         "Features": [],
#         "Model": [],
#         "Rep": [],
#         "Hyp_Name": [],
#         "Hyp_Value": []
#     }
# )



feature_results = []

auc_plot_results = []


# Areas which below pipeline will iterate through -- CHANGE THIS BASED ON NEED


#binary_outcomes = np.array(["readmission_binary", "major_med_comp", "extended_los", "returnor", "wnd_related_comp"])
#binary_outcomes = np.array(["readmission_binary"])
#binary_outcomes = np.array(["returnor", "any_med_comp_no_wnd", "any_med_comp_wnd", "any_comp_overall"])
binary_outcomes = np.array(["Post_lam_syndrome"])

#continuous_outcomes = np.array(["outpatient_resource_sum", "los_days"])
#continuous_outcomes = np.array(['los_days'])

continuous_outcomes = np.array([])


outcomes = np.concatenate((binary_outcomes, continuous_outcomes))

#procedures = ["Microdisc", "Foraminotomy", "Laminectomy", "AXDLIF", "PLF", "PTLIF"]
procedures = ["PLS"]


#features = ["Spine_Institution", "Spine_NSQIP", "Peripheral_NSQIP"]
#features = ["Spine_Institution"]
#features = ["Peripheral_NSQIP"]
features = ["PLS_features"]

model_set = ["RF", "ENet", "XGBoost", "NN", "ASA", "Dummy"]
#model_set = ["RF"]

#model_set = ["XGBoost"]
#model_set = ["Dummy"]



repetitions = list(range(5))
date = "3_11_23"

# Functions

In [13]:
def reset_random_seed(rep):
    
    tensorflow.keras.backend.clear_session()

    os.environ['PYTHONHASHSEED']=str(rep)
    os.environ['CUDA_VISIBLE_DEVICES'] = ''
    tensorflow.random.set_seed(rep)
    np.random.seed(rep)
    random.seed(rep)


In [18]:
def load_data(procedure):
    if procedure == "PLS": 
        readdata = pd.read_csv(r"081524_Spine_reports_vf.csv", sep=',')
        
    return readdata

In [ ]:
def set_columns(feature):
    if feature == 'Peripheral_NSQIP':
                                
                df = data[[

                    "caseid",
                    "age", "optime", "sex_recode", "race" , "ethnicity_hispanic", "smoke", "surgspec",
                    "bmi", "diabetes", "dyspnea", "fnstatus2", "inout", "ascites", "hxchf",
                    "hypermed", "renafail", "hxcopd", "dialysis", "wndinf", "steroid", "wtloss", 
                    "bleeddis", "transfus", "asaclas", "ventilat", "prsepis", "discancr",
                    "transt", "wndclas", "htooday", "anesthes",


                    # Outcome
                    outcome
                ]].copy()

                dum_cols = ["race", "surgspec", "fnstatus2", "asaclas", "wndclas", "anesthes", "transt"]

                numeric_features = ["age", "bmi", "optime", "htooday"]

In [ ]:
# Function for feature + outcome combo
def full_feature_outcome_test(feature, outcome):
    

In [9]:
#pip install shap==0.41.0 --user
print(shap.__version__)

0.46.0


# Code

In [19]:
# Load data

data = load_data("PLS")

/var/folders/92/tt9zk05n4qd65slhpygj9v4r0000gr/T/ipykernel_34197/2913844004.py:3: DtypeWarning: Columns (73,74,75,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,455,456,460,462,464,466,468,470,471,474,476,478,480,481,482,490,492,493,494,495,496,497,498,499,500,502,504,506,508,510,511,512,516,518,520,522,523,524,526,528,529,531,532,534,537,538,539,540,541,542,544,545,546,547,548,550,551,552,553,554,555,557,560,561,562,563,564,566,568,570,571,574,575,576,577,583,585,586,587,588,589,590,591,592,593,596,600,601,605,606,607,609,610,612,613,614,616,618,619,620,622,624,625,626,628,629,634,635,636,637,638,642,643,646,650,652,653,654,658,660,662,664,666,668,671,673,675,677,679,681,682,683,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,708,709,710,711,714,716,717,719,720,722,725,728,729,732,736,737,740,743,744,746,747,750,751,754,766,767,768,769,772,773,782,783,784,785,786,787,788,789,792,793,794,795,796,797,814,815,822,823,830,831,83

In [ ]:
full_feature_outcome_test(feature="PLS_features", outcome="Post_lam_syndrome")

In [ ]:
# 

In [ ]:
for procedure in procedures:
    
    if procedure == "LFS": 
        data = pd.read_csv(r"081524_Spine_reports_vf.csv", sep=',')
        
    for feature in features:
        
        for outcome in outcomes:
            
            if procedure == "thumb_arthro" and outcome == "returnor":
                print("skip")
                continue
                
                
            if procedure == "thumb_arthro" and outcome == "any_med_comp_no_wnd":
                print("skip")
                continue
                

            
            individual_features_shap_full = []
            
            shap_file_name = procedure+"_"+outcome+"_"+"shap_values.csv"
    

            if feature == 'Peripheral_NSQIP':
                                
                df = data[[

                    "caseid",
                    "age", "optime", "sex_recode", "race" , "ethnicity_hispanic", "smoke", "surgspec",
                    "bmi", "diabetes", "dyspnea", "fnstatus2", "inout", "ascites", "hxchf",
                    "hypermed", "renafail", "hxcopd", "dialysis", "wndinf", "steroid", "wtloss", 
                    "bleeddis", "transfus", "asaclas", "ventilat", "prsepis", "discancr",
                    "transt", "wndclas", "htooday", "anesthes",


                    # Outcome
                    outcome
                ]].copy()

                dum_cols = ["race", "surgspec", "fnstatus2", "asaclas", "wndclas", "anesthes", "transt"]

                numeric_features = ["age", "bmi", "optime", "htooday"]
                
                    
            df = df.dropna().reset_index(drop=True) # drop missing variables

            n = len(df)
            
            y = df[outcome].values

            label_n = sum(y)


            df_preds = df.drop(columns = [outcome])

            dum_df = pd.get_dummies(df_preds, columns= dum_cols)
                
            for rep in repetitions:
                                
                # Formulating the preprocessor for scaling
                scaler = preprocessing.MinMaxScaler()

                preprocessor = ColumnTransformer([('minmax', scaler, numeric_features)], remainder = 'passthrough')

                if outcome in binary_outcomes:
                    
                    skf = StratifiedKFold(n_splits=5, shuffle = True, random_state = rep)

                elif outcome in continuous_outcomes:
                    
                    print("In Progress")

                        
                
                for i, (train_index, test_index) in enumerate(skf.split(dum_df, y)):
                    
                    # Split into train and test
                    
                    x_train_split = dum_df.iloc[train_index].copy()
                    x_test_split = dum_df.iloc[test_index].copy()
                    y_train_split = y[train_index].copy()
                    y_test_split = y[test_index].copy()
                    
                    
                    # Peel off validation set
        
                    x_train_actual_split, x_val_split, y_train_actual_split, y_val_split = train_test_split(x_train_split, y_train_split, stratify = y_train_split, test_size = 0.1, random_state = rep) # random process


                    # Remove mrn and surgery start datetime and include that as the actual train and test
                        
                        
                    if feature == "Peripheral_NSQIP":
                        
                        print("Test")
                
                        x_train_actual = x_train_actual_split.drop(columns = ["caseid"])
                        x_val = x_val_split.drop(columns = ["caseid"])
                        y_train_actual = y_train_actual_split.copy()
                        y_val = y_val_split.copy()
                    
                        x_train = x_train_split.drop(columns = ["caseid"])
                        x_test = x_test_split.drop(columns = ["caseid"])
                        y_train = y_train_split.copy()
                        y_test = y_test_split.copy()
                        
                        x_train_std = preprocessor.fit_transform(x_train)

                        x_test_std = preprocessor.transform(x_test)
                
                
            
                        # Keep a version that has those values as index columns 
                    
                        x_test_features = x_test_split.copy()
                        x_test_features["Patient_ID"]=  np.arange(len(x_test_features))

                        x_test_features_long = pd.melt(x_test_features, id_vars = ["caseid", "Patient_ID"], var_name = "Feature_Name", value_name = "Feature_Actual_Value" )


                        x_cols = x_test.columns

                        n_features = len(x_cols)


            
                    for model in model_set:


                        print(procedure)
                        print(feature)
                        print(outcome)
                        print(rep)
                        print(i)
                        print(model)
                        print(time.strftime('%X %x %Z'))


                        if model == "ASA": # note - only works for binary outcomes

                            trim_train = x_train[["asaclas_2", "asaclas_3", "asaclas_4"]].copy()
                            #trim_train_reshape = trim_train.values.reshape(-1, 1)

                            trim_test = x_test[["asaclas_2", "asaclas_3", "asaclas_4"]].copy()
                            #trim_test_reshape = trim_test.values.reshape(-1, 1)

                            cv_model = linear_model.LogisticRegression(penalty = 'none', random_state = rep)
                            cv_model.fit(trim_train, y_train)

                            y_pred = cv_model.predict(trim_test)
                            y_prob = cv_model.predict_proba(trim_test)
                            y_prob_vec = y_prob[:,1]
                            #print(y_prob_vec)


                        elif model == "ASA_categorical": # note - only works for binary outcomes


                            trim_test_categorical = x_test["asa_class_above_2"]


                            y_pred = trim_test_categorical.values.reshape(-1,1)
                            y_prob = trim_test_categorical.values.reshape(-1,1)
                            y_prob_vec = trim_test_categorical.values.reshape(-1,1)
                            #print(y_prob_vec)


                        elif model == "ENet":

                            if outcome in binary_outcomes:

                                pipe = Pipeline([('processing', preprocessor), 
                                                 ('estimator', linear_model.LogisticRegression(penalty = 'elasticnet', 
                                                                                               solver='saga', 
                                                                                               class_weight = 'balanced',
                                                                                               random_state = rep, 
                                                                                               max_iter = 1000
                                                                                              ))])

                                model_params = {'estimator__C': Real(1e-2, 1e2, prior = 'log-uniform'),
                                               'estimator__l1_ratio': Real(0, 1)
                                               }

                                cv_model = BayesSearchCV(pipe, 
                                                        search_spaces = model_params, 
                                                        scoring='roc_auc', # auc is most important metric
                                                        refit=True, 
                                                        cv=5, n_jobs = 7,
                                                        n_iter = 15, 
                                                        random_state = rep
                                                        )

                                cv_model.fit(x_train, y_train)


                                y_pred = cv_model.predict(x_test)
                                y_prob = cv_model.predict_proba(x_test)
                                y_prob_vec = y_prob[:,1]


                            elif outcome in continuous_outcomes:

                                print("In Progress")



                            explainer = shap.LinearExplainer(cv_model.best_estimator_.named_steps['estimator'], x_train_std)
                            shap_values = explainer.shap_values(x_test_std)


                        elif model == "XGBoost":

                            if outcome in binary_outcomes:

                                weight_vec = wt.compute_sample_weight(class_weight = "balanced", y = y_train) # compute weights using sklearn
    #                            eval_weight = wt.compute_sample_weight(class_weight = "balanced", y = y_val)

                                pipe = Pipeline([('processing', preprocessor), 
                                                 ('estimator', xgb.XGBClassifier(#n_estimators = 500,
                                                                                 #eval_metric = "auc",
                                                                                 use_label_encoder=False,
                                                                                 verbosity = 1, random_state = rep
                                                                                ))]) 


                                es = xgb.callback.EarlyStopping(
                                    rounds = 10,
                                    save_best=True,
                                )


                                model_params = {
                                    'estimator__n_estimators': Integer(50, 250),
                                    'estimator__max_depth': Integer(2, 10),
                                    'estimator__subsample': Real(0.5, 1.0),
                                    'estimator__learning_rate': Real(0.1, 0.5),
                                    'estimator__colsample_bytree': Real(0.5, 1.0),
                                    'estimator__colsample_bylevel': Real(0.5, 1.0),
                                    'estimator__min_child_weight': Integer(1, 10),
                                    'estimator__gamma': Real(0, 1.0),
                                    'estimator__reg_lambda': Real(1e-2, 1e2, prior = 'log-uniform'),
                                    'estimator__reg_alpha': Real(1e-2, 1e2, prior = 'log-uniform'),

                                }

                                fit_params = {#'estimator__sample_weight_eval_set': [eval_weight],
    #                                          'estimator__eval_set': [(x_val_std, y_val)],
    #                                          'estimator__callbacks': [es],
                                              'estimator__sample_weight': weight_vec,
    #                                          'estimator__early_stopping_rounds': 10

                                }

                                cv_model = BayesSearchCV(pipe, 
                                                        search_spaces = model_params, n_iter = 15,
                                                        scoring='roc_auc', 
                                                        refit=True, cv= 5, 
                                                        n_jobs = 7, random_state = rep)

                                cv_model.fit(x_train, y_train, **fit_params)



                                y_pred = cv_model.predict(x_test)
                                y_prob = cv_model.predict_proba(x_test) # generate prediction probability ([0,1])
                                y_prob_vec = y_prob[:,1]



                            elif outcome in continuous_outcomes:

                                print("In Progress")




                            explainer = shap.TreeExplainer(cv_model.best_estimator_.named_steps['estimator'], data=x_train_std, model_output = "probability")
                            shap_values = explainer.shap_values(x_test_std)



                        elif model == "RF":
                            

                            if outcome in binary_outcomes:

                                pipe = Pipeline([('processing', preprocessor), 
                                                 ('estimator', BalancedRandomForestClassifier(random_state = rep))]) 


                                model_params = {'estimator__n_estimators': Integer(50, 250),
                                               'estimator__max_features': Integer(2, n_features),
                                               'estimator__max_depth': Integer(1, 25),
                                               'estimator__min_samples_split': Integer(2, 10),
                                               'estimator__min_samples_leaf': Integer(1,5),
                                               }

                                cv_model = BayesSearchCV(pipe,
                                                        search_spaces = model_params, n_iter = 15,
                                                              scoring='roc_auc', # AUC is most important metric
                                                        refit=True, cv=5, n_jobs = 7,
                                                        random_state = rep)

                                cv_model.fit(x_train, y_train)


                                y_pred = cv_model.predict(x_test) # generate prediction (0 or 1)
                                y_prob = cv_model.predict_proba(x_test) # generate prediction probability ([0,1])
                                y_prob_vec = y_prob[:,1]

                            elif outcome in continuous_outcomes:

                                print("In Progress")


                            explainer = shap.TreeExplainer(cv_model.best_estimator_.named_steps['estimator'], data=x_train_std, model_output = "probability")
                            shap_values = explainer.shap_values(x_test_std)[0]


                        elif model == "Dummy": 
                            if outcome in binary_outcomes:

                                dummy_clf = DummyClassifier(strategy="prior")
                                dummy_clf.fit(x_train, y_train)

                                y_pred = dummy_clf.predict(x_test)
                                y_prob = dummy_clf.predict_proba(x_test)
                                y_prob_vec = y_prob[:,1]

                            elif outcome in continuous_outcomes:

                                dummy_clf = DummyRegressor(strategy="median")
                                dummy_clf.fit(x_train, y_train)

                                y_pred = np.round(dummy_clf.predict(x_test))

                        elif model == "NN":


                            if outcome in binary_outcomes:

                                x_train_actual_std = preprocessor.fit_transform(x_train_actual)
                                x_val_std = preprocessor.transform(x_val)

    #                            x_train_std = preprocessor.fit_transform(x_train)
                                x_test_std = preprocessor.transform(x_test)

                                reset_random_seed(rep)


                                metric_set=[tensorflow.keras.metrics.AUC(name="auc")]



                                weight_vec = wt.compute_sample_weight(class_weight = "balanced", y = y_train_actual) # compute weights using sklearn
                                eval_weight = wt.compute_sample_weight(class_weight = "balanced", y = y_val)



                                nn_model = models.Sequential(name="DeepNN_CLF", layers=[

                                    # Input layer is implicitly defined here

                                    ### hidden layer 1
                                    layers.Dense(name="h1", input_dim=n_features,
                                                 units=int(round((n_features+1)/2)), 
                                                 activation='relu'),

                                    ### dropout layer 1
                                    layers.Dropout(name="drop1", rate=0.2, seed = rep),

                                    ### hidden layer 2
                                    layers.Dense(name="h2", units=int(round((n_features+1)/4)), 
                                                 activation='relu'),

                                    ### dropout layer 2
                                    layers.Dropout(name="drop2", rate=0.2, seed = rep),

                                    ### layer output
                                    layers.Dense(name="output", units=1, activation='sigmoid')
                                ])

                                nn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics = metric_set)

                                early_stopping = callbacks.EarlyStopping(
                                    monitor='val_auc', 
                                    verbose=1,
                                    patience=5,
                                    mode='max',
                                    restore_best_weights=True)

                                nn_model.fit(
                                    x_train_actual_std,
                                    y_train_actual,
                                    sample_weight = weight_vec,
                                    epochs= 100,
                                    callbacks=[early_stopping],
                                    validation_data=(x_val_std, y_val, eval_weight)
                                )

                                y_pred = (nn_model.predict(x_test_std) > 0.5).astype("int32")                 
                                y_prob = nn_model.predict(x_test_std) 
                                y_prob_vec = y_prob[:,0]

                                nn_model.summary()


                            elif outcome in continuous_outcomes:

                                print("In Progress")



                            explainer = shap.DeepExplainer(nn_model, data=x_train_actual_std)
                            shap_values = explainer.shap_values(x_test_std)[0]



                        # Computing Results regardless of model

                        if outcome in binary_outcomes:
                            # Computing AUC
                            fpr, tpr, threshold = metrics.roc_curve(y_test, y_prob_vec, pos_label = 1)
                            roc_auc = metrics.auc(fpr, tpr)
                            AUC = metrics.auc(fpr, tpr)

                            print(AUC)
                            # Sensitivity, Specificity

                            recall_score_0 = metrics.recall_score(y_test,y_pred, pos_label=0)
                            recall_score_1 = metrics.recall_score(y_test,y_pred, pos_label=1)

                            # PPV, NPV
                            precision_score_0 = metrics.precision_score(y_test,y_pred, pos_label=0)
                            precision_score_1 = metrics.precision_score(y_test,y_pred, pos_label=1)

                            # Computing Log Loss
                            log_loss = metrics.log_loss(y_test, y_prob_vec, labels = [0,1])

                            # Computing Brier score
                            brier_loss = metrics.brier_score_loss(y_test, y_prob_vec, pos_label = 1)

                            # Computing WF1
                            W_F1= metrics.f1_score(y_test, y_pred, average='weighted')

                            # Computing Accuracy
                            accuracy_score = metrics.accuracy_score(y_test,y_pred)
                            B_accuracy_score = metrics.balanced_accuracy_score(y_test,y_pred)


                            # Confusion Matrix
                            c_mat = metrics.confusion_matrix(y_test, y_pred)

                            new_results = pd.DataFrame(
                                {
                                    "Procedure": [procedure],
                                    "Outcome": [outcome],
                                    "Features": [feature],
                                    "Model": [model],
                                    "Rep": [rep],
                                    "Fold": [i],
                                    "W_F1": [W_F1],
                                    "AUC": [AUC],
                                    "MAE": ["NA"],
                                    "R2": ["NA"],
                                    "RMSE": ["NA"],
                                    "Log-Loss": [log_loss],
                                    "Brier-Loss": [brier_loss],
                                    "Accuracy": [accuracy_score],
                                    "Balanced_Accuracy": [B_accuracy_score],
                                    "Sensitivity": [recall_score_1],
                                    "Specificity": [recall_score_0],
                                    "NPV": [precision_score_0],
                                    "PPV": [precision_score_1],
                                    "c_matrix": [c_mat],
                                    "n":[n],
                                    "label_n": [label_n]
                                }
                            )

                            auc_plot_results_new = pd.DataFrame(
                                {
                                    "Procedure": np.repeat(procedure, len(fpr)),
                                    "Outcome": np.repeat(outcome, len(fpr)),
                                    "Features": np.repeat(feature, len(fpr)),
                                    "Model": np.repeat(model, len(fpr)),
                                    "Rep": np.repeat(rep, len(fpr)),
                                    "Fold": np.repeat(i, len(fpr)),
                                    "FPR": fpr,
                                    "TPR": tpr,
                                    "Thresholds": threshold
                                }
                            )

                            auc_plot_results.append(auc_plot_results_new)


                        elif outcome in continuous_outcomes:
                            print("In Progress")


                        all_results.append(new_results)

                        if model == "Dummy":
                            continue

                        if model == "ASA" or model == "ASA_categorical": 
                            continue

                        mod_shap = shap_values.copy()
                        #mod_shap = shap_values[0]

                        test_length = len(y_test)

                        row_vec = list(range(0, test_length))
                        shap_features_row = []

                        for row in row_vec:

                            shap_values_row = mod_shap[row]

                            temp_feature_names = list(x_cols)
                            



                            new_shap_features_row = pd.DataFrame(
                                {
                                    "Patient_ID": np.repeat(row, len(x_cols)),
                                    "Feature_Name": temp_feature_names,
                                    "Feature_Value": shap_values_row,
                                    "Feature_Abs_Value": np.abs(shap_values_row),

                                }
                            )

                            shap_features_row.append(new_shap_features_row)




                        shap_features_tabular = pd.concat(shap_features_row)



                        individual_features_shap = x_test_features_long.merge(shap_features_tabular, how = "left", on = ["Patient_ID", "Feature_Name"])

                        individual_features_shap["Procedure"]=  procedure
                        individual_features_shap["Outcome"]=  outcome
                        individual_features_shap["Model"]=  model
                        individual_features_shap["Rep"]=  rep
                        individual_features_shap["Fold"]=  i


                        individual_features_shap_full.append(individual_features_shap)

                        shap_file_results = pd.concat(individual_features_shap_full, ignore_index = True)
                        shap_file_results.to_csv(shap_file_name)  


                        summarized_features_shap = shap_features_tabular.groupby(['Feature_Name']).agg(   
                            Feature_Mean_Abs = ('Feature_Abs_Value','mean'),
                        ).reset_index()


                        feature_coef = summarized_features_shap['Feature_Mean_Abs'].values
                        feature_names = summarized_features_shap['Feature_Name'].values


                        new_features = pd.DataFrame(
                            {
                                "Procedure": np.repeat(procedure, len(feature_names)),
                                "Outcome": np.repeat(outcome, len(feature_names)),
                                "Features": np.repeat(feature, len(feature_names)),
                                "Model": np.repeat(model, len(feature_names)),
                                "Rep": np.repeat(rep, len(feature_names)),
                                "Fold": np.repeat(i, len(feature_names)),
                                "Feature_Name": feature_names,
                                "Feature_Mean_Abs_Value": feature_coef,

                            }
                        )


    #                    feature_results = feature_results.append(new_features, ignore_index=True)
                        feature_results.append(new_features)



    #                    all_results = all_results.append(new_results, ignore_index=True)



    
    file_name_base = procedure+"_"+date+"_"
    
    output_file_results = pd.concat(all_results, ignore_index = True)
    output_file_results = output_file_results[output_file_results['Procedure'] == procedure]
    output_file_results.to_csv(file_name_base+"_"+"model_results.csv")
    
    
    feature_file_results = pd.concat(feature_results, ignore_index = True)
    feature_file_results = feature_file_results[feature_file_results['Procedure'] == procedure]
    feature_file_results.to_csv(file_name_base+"_"+"feature_importance.csv")    
    
    
    #print(auc_plot_results)
    auc_file_results = pd.concat(auc_plot_results, ignore_index = True)
    auc_file_results = auc_file_results[auc_file_results['Procedure'] == procedure]
    auc_file_results.to_csv(file_name_base+"_"+"auc_data.csv") 
    



print(time.strftime('%X %x %Z'))




In [ ]:
feature_results_output = pd.concat(feature_results, ignore_index = True)
feature_results_output.to_csv("thumb_arthro_features_3_11_23_partial.csv")

In [ ]:
#print(len(np.repeat(row, len(x_cols)))
print(len(shap_values_row))
print(len(temp_feature_names))

In [ ]:
print(all_results)

In [ ]:
individual_features_shap.to_csv("tester_shap.csv")

In [ ]:
    output_file_results.to_csv(file_name_results)
    
    
    feature_file_results = pd.concat(feature_results)
    feature_file_results = feature_file_results[feature_file_results['Procedure'] == procedure]
    feature_file_results.to_csv(file_name_features) 

In [ ]:
auc_plot_results.to_csv("auc_results_health_util_11_7_22.csv")    

In [ ]:
print(output_file_results)
np.median(output_file_results['MAE'])

In [ ]:
print(shap_features_row)

#shap_features_row.to_csv("NN_feature_test.csv")

summarized_features_shap = shap_features_row.groupby(['Feature_Name']).agg(
    Feature_Mean_Abs = ('Feature_Value_Abs','mean'),   
    Feature_Mean_Real = ('Feature_Value_Real','mean') 
).reset_index()

print(summarized_features_shap)
print()

In [ ]:
pip install joblib==1.1.0 --user

In [ ]:
#print(feature_results)
#print(all_results)
#feature_results.to_csv("feature_test_1_30_21.csv")
partial_output_results = pd.concat(all_results)
partial_output_features = pd.concat(feature_results)
partial_output_results.to_csv("partial_acdf_results_5_31_22.csv")
partial_output_features.to_csv("partial_acdf_features_5_31_22.csv")
#hyperparam_results.to_csv("all_results_acdf_cbc_bmp_hyperparam_lasso_brf_xgb_2_6_22_updated.csv")

print(feature_results)

In [ ]:
partial_output_file_results = pd.concat(all_results)
partial_output_file_results = partial_output_file_results[partial_output_file_results['Procedure'] == "PLF"]
partial_output_file_results.to_csv("partial_plf_results_5_31_22.csv")
    
    
partial_feature_file_results = pd.concat(feature_results)
partial_feature_file_results = partial_feature_file_results[partial_feature_file_results['Procedure'] == "PLF"]
partial_feature_file_results.to_csv("partial_plf_features_5_31_22.csv")   

In [ ]:
print(n_features)

In [ ]:
print(outcome)
print(procedure)
print(feature)

In [ ]:
summarized_results = all_results.groupby(['Outcome', 'Features', 'Model']).agg(
    WF1_Mean = ('W_F1','mean'),
    WF1_SD = ('W_F1', 'std'),
    AUC_Mean = ('AUC', 'mean'),
    AUC_SD = ('AUC', 'std'),
    Acc_Mean = ('Accuracy', 'mean'),
    Acc_SD = ('Accuracy', 'std'),
    BAcc_Mean = ('Balanced_Accuracy', 'mean'),
    BAcc_SD = ('Balanced_Accuracy', 'std'),
    Log_Loss_Mean = ('Log-Loss', 'mean'),
    Log_Loss_SD = ('Log-Loss', 'std'),
    Precision_Neg_Mean = ('Precision_Neg', 'mean'),
    Precision_Neg_SD = ('Precision_Neg', 'std'),
    Recall_Neg_Mean = ('Recall_Neg', 'mean'),
    Recall_Neg_SD = ('Recall_Neg', 'std'),
    Precision_Pos_Mean = ('Precision_Pos', 'mean'),
    Precision_Pos_SD = ('Precision_Pos', 'std'),
    Recall_Pos_Mean = ('Recall_Pos', 'mean'),
    Recall_Pos_SD = ('Recall_Pos', 'std'),
    n_Mean = ('n', 'mean'),
    n_SD = ('n', 'std'),
    label_n_Mean = ('label_n', 'mean'),
    label_n_SD = ('label_n', 'std')
    
    
).reset_index()

summarized_results.to_csv("wnd_dehis_summarized_lasso_brf_xgb_2_6_22_updated.csv")

print(summarized_results)

In [ ]:
print(feature_coef)
print(feature_coef_real)

In [ ]:
#feature_dat = pd.read_csv(r"Results/LASSO/pre_op_features_2_1_22.csv", sep=',')
feature_dat = pd.read_csv(r"wnd_dehis_features_lasso_brf_xgb_2_3_22_updated.csv", sep=',')


feature_dat['Feature_Value_Real'] = feature_dat['Feature_Value_Real'].str.strip('[]').astype(float) # uncomment and run this with lasso

summarized_features = feature_dat.groupby(['Outcome', 'Model', 'Feature_Name']).agg(
    Feature_Value_Mean = ('Feature_Value_Real', 'mean'),
    Feature_Value_SD = ('Feature_Value_Real', 'std'),
    
).reset_index()

summarized_features.to_csv("wnd_dehiscence_xgb_features_summarized_2_3_22_updated.csv")